In [ ]:
from pip._internal import main as pip

try:
    import requests
except ImportError:
    pip(['install', 'request'])
    import requests
    
try:
    import urllib
except ImportError:
    pip(['install', 'urllib'])
    import urllib

try:
    import pandas
except ImportError:
    pip(['install', 'pandas'])
    import pandas

try:
    from IPython.display import display, SVG, HTML, Image
except ImportError:
    pip(['install', 'IPython'])
    from IPython.display import display, SVG, HTML
    



In [180]:
url = 'https://lazar.prod.openrisknet.org/authenticate/login'
headers = {'accept': 'application/json', 'Content-Type': 'application/x-www-form-urlencoded'}
payload = {'username':'name', 'password': 'pw'}

r = requests.post(url, headers=headers, data=payload)
if r.status_code == 200:
    res = r.json()
    subjectid = res['subjectid']
else:
    print(r.json())

url = 'https://lazar.prod.openrisknet.org/endpoint'
headers = {'accept': 'application/json',
           'Content-Type': 'application/x-www-form-urlencoded',
           'subjectid': subjectid
          }

r1 = requests.get(url, headers=headers)

print("LAZAR Status code GET endpoints: {0}".format(r1.status_code))
if r1.status_code == 200:
    endpoints = r1.json()

LAZAR Status code GET endpoints: 200


In [181]:
data = []

def extractSpecies(speciesList):
    for item in speciesList:
        for species, url in item.items():
            yield species, url            
            
for endpoint in endpoints:
    url = 'https://lazar.prod.openrisknet.org/endpoint/' + endpoint
    r2 = requests.get(url, headers=headers);

    if r2.status_code == 200:
        for species, url in extractSpecies(r2.json()):
            data.append([endpoint, species, url])

df = pandas.DataFrame(data, columns=["Endpoint", "Species", "URL"])

df


,Endpoint,Species,URL
0,Blood Brain Barrier Penetration,Human,https://lazar.prod.openrisknet.org/model/5ae2dd885f1c2d01323270ee
1,Carcinogenicity,Rat,https://lazar.prod.openrisknet.org/model/5ae2de075f1c2d0132327658
2,Carcinogenicity,Rodents (multiple species/sites),https://lazar.prod.openrisknet.org/model/5ae2e2425f1c2d0132328730
3,Carcinogenicity,Mouse,https://lazar.prod.openrisknet.org/model/5ae2e2a55f1c2d0132328867
4,Carcinogenicity,Mouse (TD50),https://lazar.prod.openrisknet.org/model/5ae2ff635f1c2d0132329a54
5,Carcinogenicity,Rat (TD50),https://lazar.prod.openrisknet.org/model/5ae3034f5f1c2d0132329c82
6,Mutagenicity,Salmonella typhimurium,https://lazar.prod.openrisknet.org/model/5ae2e1cd5f1c2d0132328594
7,Lowest observed adverse effect level (LOAEL),Rat,https://lazar.prod.openrisknet.org/model/5ae2e9225f1c2d0132328d67
8,Acute toxicity,Daphnia magna,https://lazar.prod.openrisknet.org/model/5ae2eda05f1c2d0132328fde
9,Acute toxicity,Fathead minnow,https://lazar.prod.openrisknet.org/model/5ae3097a5f1c2d0132329f40


In [182]:
smiles = 'CCCCCCCC(=O)OC(COC(=O)CCCCCCC)COC(=O)CCCCCCC'

print('SMILES string we want a prediction for: ' + smiles )

SMILES string we want a prediction for: CCCCCCCC(=O)OC(COC(=O)CCCCCCC)COC(=O)CCCCCCC


In [183]:
selector = 7

In [184]:
print("Nr. " + str(selector) + " selected")
endpoint = df['Endpoint'][selector]
species = df['Species'][selector]
model = df['URL'][selector]
print('\n'.join([endpoint,species,model]))
r3 = requests.post(df['URL'][selector], 
                   data={'identifier': smiles},
                   headers=headers);

print("\nLAZAR Status code POST: {0}".format(r3.status_code))
if r3.status_code == 200:
    prediction = r3.json()

Nr. 7 selected
Lowest observed adverse effect level (LOAEL)
Rat
https://lazar.prod.openrisknet.org/model/5ae2e9225f1c2d0132328d67

LAZAR Status code POST: 200


In [185]:
#print(prediction)
if 'probabilities' in prediction['prediction']:
    type = "classification"
else: 
    type = "regression"


data = []
pandas.set_option('display.max_colwidth', -1)
unit = prediction['model']['unit']
if not unit:
    unit = ''

# collect only main values
pred = {}
pred['warnings'] = prediction['prediction']['warnings']
if prediction['prediction'].get('info'):
    pred['info'] = prediction['prediction']['info']
pred['measurements'] = prediction['prediction']['measurements']
pred['value'] = prediction['prediction']['value']
if type == 'classification':
    if prediction['prediction'].get('probabilities'):
        pred['probabilities'] = prediction['prediction']['probabilities']
else:
    if prediction['prediction'].get('prediction_interval'):
        pred['prediction_interval'] = prediction['prediction']['prediction_interval']

for p in pred:
    # don't show empty info or warning
    if pred.get(p):
        if isinstance(pred.get(p), list):
            arr = pred.get(p)
            a = []
            for v in arr:
                # readout array; delog10 float values; 
                # measurements; prediction_interval
                if isinstance(v, float):
                    s = str(10**(-1*v))
                    a.append(s)
            if a:
                string = ','.join(a)
                data.append([p,string,unit])
            else:
                data.append([p,','.join(arr),''])
        # readout hash; delog10 float values; probabilities
        elif isinstance(pred.get(p), dict):
            d = pred.get(p)
            a = []
            for k in d:
                string = ''
                string += k + ': '
                string += str(d[k])
                a.append(string)
            out = ','.join(a)
            data.append([p,out,unit])
        else:
            v = pred.get(p)
            if isinstance(v, float):
                # regression prediction value
                data.append([p,str(10**(-1*v)),unit])
            else:
                # classification prediction value
                data.append([p,v,''])

if unit:
    result = pandas.DataFrame(data, columns=['Name','Value', 'Unit'])
else:
    for d in data:
        del d[2::2]
    result = pandas.DataFrame(data, columns=['Name','Value'])

print("LAZAR " + type + " prediction result for: " + smiles)
print('\n'.join([endpoint,species,model]))

result

LAZAR regression prediction result for: CCCCCCCC(=O)OC(COC(=O)CCCCCCC)COC(=O)CCCCCCC
Lowest observed adverse effect level (LOAEL)
Rat
https://lazar.prod.openrisknet.org/model/5ae2e9225f1c2d0132328d67


,Name,Value,Unit
0,value,8.97040156372302,mmol/kg_bw/day
1,prediction_interval,"82.09250047007968,0.9802126108190927",mmol/kg_bw/day


In [186]:
pandas.set_option('display.max_colwidth', -1)

if prediction['prediction'].get('neighbors'):
    prediction_neighbors = pandas.DataFrame(prediction['prediction']['neighbors']).sort_values(['similarity'], ascending=False)
    #print(prediction_neighbors)
    prediction_neighbors.index = range(0,len(prediction_neighbors))
    #print(prediction_neighbors)

    df_neighbors = pandas.DataFrame(columns=['Image', 'SMILES', 'Measurement', 'Similarity'])

    idx = 0
    for id in prediction_neighbors['id']:
        compoundreq = requests.get('https://lazar.prod.openrisknet.org/substance/'+id, headers=headers)
        compound = compoundreq.json()
        c = compound['compound']
        smiles = c['smiles']
        inchi = c['inchi']
        esc_inchi = urllib.parse.quote(inchi)
        req=requests.get('https://lazar.prod.openrisknet.org/compound/'+esc_inchi, headers={'accept':'image/svg+xml', 'subjectid': subjectid})
        svg=req.content
        df_neighbors = df_neighbors.append({'Image': svg, 'SMILES': smiles, 'Measurement': prediction_neighbors.at[idx, 'measurement'], 'Similarity': prediction_neighbors.at[idx, 'similarity']}, ignore_index=True)
        idx = idx+1

    print("Neighbors:\n")

    display(HTML(df_neighbors.to_html(escape=False, formatters=dict(status=svg))))
    
else:
    print('No neighbors available')


Neighbors:

